In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# shape 변환
x_train = x_train[:,:,:,tf.newaxis]
x_val = x_val[:,:,:, tf.newaxis]

In [ ]:
x_train.shape, x_val.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [ ]:
# 자료형 변환 # tf.cast 자료형 변환
x_train = tf.cast(x_train, tf.float32)
y_train = tf.cast(y_train, tf.float32)

x_val = tf.cast(x_val, tf.float32)
y_val = tf.cast(y_val, tf.float32)

In [ ]:
# y값 one-hot encoding
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

In [ ]:
# tf data 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [ ]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [ ]:
# map 
# x와 y가 세트로 묶여있기 때문에 y를 처리하지 않더라도 같이 넣어줘야 합니다.
def norm(x_data, y_data):
    x_data = x_data / 255.0
    return x_data, y_data

In [ ]:
# map 적용
train_dataset = train_dataset.map(norm)

In [ ]:
# 데이터셋 섞기
buffer_size = len(x_train) # 얼만큼 섞을 것인가, 일반적으로 데이터 길이만큼
train_dataset = train_dataset.shuffle(buffer_size)

In [ ]:
# batch size만큼 나누기
batch_size = 64
train_dataset = train_dataset.batch(batch_size) 

In [ ]:
# repeat -> 반복, 한번더
train_dataset = train_dataset.repeat()

In [ ]:
# 데이터 미리 준비시키기
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for x,y in train_dataset.take(1):
    print(x)

In [ ]:
# val set 만들기
val_dataset = tf.data.Dataset.from_tensor_slices((x_val,y_val))
val_dataset = val_dataset.shuffle(buffer_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
# model
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# compile
model.compile(
    optimizer="adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# model fit
model.fit(
    train_dataset, # train_dataset
    epochs=10,
    validation_data=val_dataset, # validation_dataset
    steps_per_epoch= len(x_train) / batch_size   # steps_per_epoch
    # batch_size 넣어주지 마세요, 배치단위로 알아서 들어갑니다
)

Epoch 1/10
937/937 [==============================] - 7s 5ms/step - loss: 0.3388 - accuracy: 0.9046 - val_loss: 24.3856 - val_accuracy: 0.9488
Epoch 2/10
937/937 [==============================] - 5s 4ms/step - loss: 0.1558 - accuracy: 0.9549 - val_loss: 19.6112 - val_accuracy: 0.9611
Epoch 3/10
937/937 [==============================] - 5s 4ms/step - loss: 0.1159 - accuracy: 0.9658 - val_loss: 17.9280 - val_accuracy: 0.9641
Epoch 4/10
937/937 [==============================] - 5s 4ms/step - loss: 0.0939 - accuracy: 0.9718 - val_loss: 16.9240 - val_accuracy: 0.9679
Epoch 5/10
937/937 [==============================] - 4s 3ms/step - loss: 0.0805 - accuracy: 0.9754 - val_loss: 19.4028 - val_accuracy: 0.9636
Epoch 6/10
937/937 [==============================] - 5s 4ms/step - loss: 0.0663 - accuracy: 0.9791 - val_loss: 16.9975 - val_accuracy: 0.9702
Epoch 7/10
937/937 [==============================] - 4s 3ms/step - loss: 0.0569 - accuracy: 0.9820 - val_loss: 18.9348 - val_accuracy: 0.9705

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle("naver_stock.pkl")
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2021-08-19,424500.0,4500.0,431000.0,434000.0,424500.0,501090.0
1,2021-08-18,429000.0,500.0,429500.0,433500.0,421500.0,496576.0
2,2021-08-17,428500.0,8000.0,435500.0,437500.0,420000.0,763564.0
3,2021-08-13,436500.0,4000.0,445000.0,446000.0,433000.0,515855.0
4,2021-08-12,440500.0,4500.0,448000.0,448500.0,438000.0,671083.0


In [ ]:
x_col = "시가 고가 저가 거래량".split(" ")
y_col = ["종가"]

In [ ]:
x = df[x_col]
y = df[y_col]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x,y))

In [ ]:
dataset = dataset.batch(32)

In [ ]:
for x,y in dataset.take(1):
    print(x)

tf.Tensor(
[[ 431000.  434000.  424500.  501090.]
 [ 429500.  433500.  421500.  496576.]
 [ 435500.  437500.  420000.  763564.]
 [ 445000.  446000.  433000.  515855.]
 [ 448000.  448500.  438000.  671083.]
 [ 445000.  449000.  442000.  446275.]
 [ 446000.  453500.  442500.  461940.]
 [ 438000.  449000.  435000.  528032.]
 [ 443000.  447500.  440500.  416829.]
 [ 442000.  451500.  438500.  744924.]
 [ 425000.  434500.  423000.  545873.]
 [ 439000.  439000.  422000.  830841.]
 [ 429000.  437000.  428500.  551937.]
 [ 438000.  440500.  433000.  628812.]
 [ 444000.  445000.  438500.  556532.]
 [ 446000.  447000.  440000.  913055.]
 [ 457500.  463000.  452000.  757638.]
 [ 460000.  465000.  447500.  855277.]
 [ 443500.  454500.  441000. 1094026.]
 [ 433000.  445000.  429000. 1035882.]
 [ 443000.  446000.  428000.  885519.]
 [ 438500.  441500.  431000.  789090.]
 [ 443000.  449500.  438500.  704307.]
 [ 448000.  461000.  443000. 1246425.]
 [ 446500.  452500.  439500.  950460.]
 [ 436000.  44